# Retrieval API DEMO

## Code Initialisation
Dependencies and environment initialisation.

Ensure there's a `.env` file with your credentials in the same directory as this script. Use the `env` file as template.

In [1]:
import os
import json
import requests as r
from IPython.display import Markdown
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [2]:
API_HOST = 'api.dowjones.com'
AUTH_HOST = 'accounts.dowjones.com'
CLIENT_ID = os.getenv('FACTIVA_CLIENTID')
USERNAME = os.getenv('FACTIVA_USERNAME')
PASSWORD = os.getenv('FACTIVA_PASSWORD')
AUTH_URL = f"https://{AUTH_HOST}/oauth2/v1/token"
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_LOCATION = os.getenv('GCP_LOCATION')
GCP_MODEL = os.getenv('GCP_MODEL')

## Authentication - Generate Bearer

For details about getting the `bearer_token`, please see the `utils.py` file.

In [3]:
bearer_token = u.get_bearer_token(CLIENT_ID, USERNAME, PASSWORD, AUTH_URL)
if bearer_token:
    display(Markdown(f"**Authentication Successful**: Bearer token OK for user {USERNAME.split('@')[0].split('-')[0]}"))
else:
    display(Markdown(f"**Authentication Failed**: Cannot obtain the Bearer token for the user {USERNAME.split('@')[0].split('-')[0]}"))

**Authentication Successful**: Bearer token OK for user 9ZZZ159100

In [4]:
req_headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}

## Factiva Retrieval API Query

### Prompt
Add the prompt to be sent to the Retrieval API

In [ ]:
# "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?"
# "中国2025年计划进行哪些月球探测任务？其主要目标是什么？"   # What are China's planned missions to the Moon in 2025, and what are their primary objectives?
# "今後12か月間の日本の半導体業界の見通しはどのようなものですか?"  # What are the perspectives for the semiconductors industry in Japan for the next 12 months?
# "Summarise the latest earnings report from Microsoft Corp"
# "What are the perspectives of bitcoin for the next 12 months?"
frapi_prompt = "Summarise tariff decisions by the US administration during the first week of April 2025 and their impact on the world economy"

Assemble the Retrieval API payload

In [20]:
frapi_query = {
  "data": {
    "attributes": {
      "response_limit": 10,
      "query": {
        "search_filters": [
          {
            "scope": "Language",
            "value": "en"       # en or ja or de or it or es
          }
        ],

        "value": frapi_prompt
      }
    },
    "id": "GenAIRetrievalExample",
    "type": "genai-content"
  }
}


## Send Query and Receive Chunks from the Retrieval API

In [21]:
chunks_resp = r.post(f"https://{API_HOST}/content/gen-ai/retrieve", json=frapi_query, headers=req_headers)

if chunks_resp.status_code == 200:
    print('Successfully retrieved chunks')
else:
    print(f"Request Failed: {chunks_resp.json()}")

Successfully retrieved chunks


## Print Chunks

In [22]:
chunks_list = chunks_resp.json()['data']

# u.print_full_chunks(chunks_list)
u.print_partial_chunks(chunks_list)

### Trump announces pause in 'reciprocal' tariffs for all countries but China 

**Washington Post.com** - 2025-04-09 - WPCOM00020250409el490018j - Lang: en

President Donald Trump on Wednesday raised tariffs on imports from China to 125 percent while simultaneously pausing many new tariffs on goods from ot...

---

### Stocks sink despite talk of trade deals

...

---

### Trump Reciprocal Tariffs Take Effect, Upending U.S. Trade -- WSJ

**Dow Jones Institutional News** - 2025-04-09 - DJDN000020250409el4900055 - Lang: en

President Trump imposed more tariffs on nearly 100 nations, hiking U.S. duties to the highest level since before World War II and sparking a race amon...

---

## Retrieval API Conclusion

Up to this point it the seen functionalities are connected with the Retrieval API.

However, given that this service is an intermediate component in a full-stack solution, the below Test LLM steps and the [Read Article](2_read_article.ipynb) notebook are the two complementary avenues to get a full working solution.

## Test LLM

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example only a few articles are used to respond the prompt using an LLM hosted in **Google Cloud Vertex AI**. The passed request is built from the original prompt enhanced with the retrieved articles as gounding context.

The tested LLM is Google Gemini 2.0 Flash. Response generations took a few seconds for a context between 3K to 4K LLM tokens.

### Prerequisite

To successfully complete the following steps, it's important that there's an active GCloud authentication in the environment where this notbeook is executed from. This can be done by running the following command:

```bash
$ gcloud auth application-default login
```

For more information see [Set up ADC for a local development environment](https://cloud.google.com/docs/authentication/set-up-adc-local-dev-environment).

### Gemini Structured Prompt

In [23]:
instructions_text ="""
    You are an experienced business analyst that respond in a professional manner.
    Answer the query using only the information provided in the list of articles.
    If you use information from an article, cite it using squared brackets containing the index number.
    At the end of the answer, show a list of the cited articles ordered by index and under the title Cited Articles.
    Each cited article must be displayed in the following Markdown format:
    - [index] [headline - source_name - publication_date](url)
    Use Markdown for the output.
"""
article_list = []

for chunk in chunks_list:
    article = {
        'index': len(article_list) + 1,
        'url': f"https://dj.factiva.com/article?id=drn:archive.newsarticle.{str(chunk['meta']['original_doc_id']).strip()}",
        'source_name': str(chunk['meta']['source']['name']).strip(),
        'headline': str(chunk['attributes']['headline']['main']['text']).strip(),
        'publication_date': chunk['attributes']['publication_date'],
        'content': f"{str(chunk['attributes']['snippet']['content'][0]['text']).strip()} {str(chunk['attributes']['content'][0]['text']).strip()}"
    }
    article_list.append(article)

llm_prompt = {
    'query': frapi_prompt,
    'articles': article_list,
    "instructions": instructions_text.strip()
}

In [25]:
# u.print_full_llm_prompt(llm_prompt)
u.print_partial_llm_prompt(llm_prompt)

{
    "query": "Summarise tariff decisions by the US administration during the first week of April 2025 and their impact on the world economy.",
    "articles": [
        {
            "index": 1,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.WPCOM00020250409el490018j",
            "source_name": "Washington Post.com",
            "headline": "Trump announces pause in 'reciprocal' tariffs for all countries but China",
            "publication_date": "2025-04-09",
            "content": "President Donald Trump on Wednesday raised tariffs on imports from China to 125 percent while simultaneously pausing many new tariffs on goods from ot..."
        },
        {
            "index": 2,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.WP00000020250409el4900013",
            "source_name": "The Washington Post",
            "headline": "Stocks sink despite talk of trade deals",
            "publication_date": "2025-04-09",
       

### Gemini Request

In [26]:
response = u.gemini_generate(llm_prompt, GCP_PROJECT, GCP_LOCATION, GCP_MODEL)
display(Markdown(f"**Prompt:** {frapi_prompt} \n --------------------"))
display(Markdown(response))

**Prompt:** Summarise tariff decisions by the US administration during the first week of April 2025 and their impact on the world economy. 
 --------------------

During the first week of April 2025, the US administration under President Donald Trump implemented significant tariff decisions, primarily impacting trade relations with China and other nations.

On April 2nd, Trump announced a 10% tariff on all goods, plus an additional tailored punitive import tax applying to dozens of countries [8]. By April 5th, the U.S. started collecting a 10% 'baseline' tariff on all goods [8]. On April 7th, Trump announced an additional 50% tariff on China [8].

On April 9th, these tariffs came into effect for 86 countries, raising the duty on all Chinese goods to 104% [8]. Including prior levies, the average tariff rate on China is estimated at around 125% [10]. China retaliated by imposing an 84% levy on all U.S. goods [5]. This tit-for-tat escalation raised fears of a global economic downturn [5]. Trump also paused many new tariffs on goods from other nations [1]. Sector-specific tariffs being considered by the White House, on industries such as pharmaceutical and lumber imports, would not be affected by the pause [1].

The impact on the world economy was substantial. The stock market experienced significant volatility [1, 4]. On April 3rd, U.S. markets suffered their steepest declines since 2020, with the S&P 500 falling 4.8% [4]. China retaliated with 34% additional tariffs on all goods imported from the U.S., and the S&P 500 plunged an additional 6% [4]. European markets also sold off, with London's FTSE 100, France's CAC index, and Germany's DAX each down roughly 3 percent [1]. In Asia, Japan's Nikkei 225 lost 4 percent, while South Korea's KOSPI ended the day roughly 2 percent lower [1]. Hong Kong's Hang Seng Index fell sharply but recovered to eke out a modest gain [1].

JPMorgan raised the probability of a global recession to 60% from 40% [4]. The bank expects U.S. gross domestic product to contract 0.3% in the fourth quarter of 2025 from a year earlier; it previously expected 1.3% growth [4]. Unemployment will reach 5.3% next year, JPMorgan says [4]. Nomura Chief Economist David Seif predicts GDP will grow 0.6% in the fourth quarter from a year earlier [4].

Walmart warned there is more uncertainty about how fast its profits will grow in the first quarter, citing President Donald Trump's latest barrage of tariffs [6].

The tariffs are expected to raise the average effective U.S. tariff rate from 2.5% in 2024 to around 22.5%, according to the left-of-center Yale Budget Lab [4]. Combined with narrower tariffs Trump imposed in February and March, that will push up prices by 2.3% in the short run, equivalent to $3,800 less purchasing power for the average household [4].

China vowed to take countermeasures with "firm will" and "abundant means" following U.S. tariff hikes, while urging dialogue to resolve respective concerns and stabilize bilateral ties [3].

Cited Articles:
- [1] [Trump announces pause in 'reciprocal' tariffs for all countries but China - Washington Post.com - 2025-04-09](https://dj.factiva.com/article?id=drn:archive.newsarticle.WPCOM00020250409el490018j)
- [3] [4th LD-Writethru-China Focus: China vows firm countermeasures following U.S. tariff moves, urges dialogue - Xinhua News Agency - 2025-04-09](https://dj.factiva.com/article?id=drn:archive.newsarticle.XNEWS00020250409el490083p)
- [4] [The Outlook: Tariffs Erase Economy's Solid Outlook -- WSJ - Dow Jones Institutional News - 2025-04-07](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250407el47000dg)
- [5] [China increases levies on U.S. as Trump's 104% tariffs take effect; stocks sink - Washington Post.com - 2025-04-09](https://dj.factiva.com/article?id=drn:archive.newsarticle.WPCOM00020250409el4900001)
- [6] [Walmart: Tariffs Increase Uncertainty on Profit Growth This Quarter -- Barrons.com - Dow Jones Institutional News - 2025-04-09](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250409el490031a)
- [8] [See all the tariffs Trump has enacted, threatened and canceled - Washington Post.com - 2025-04-09](https://dj.factiva.com/article?id=drn:archive.newsarticle.WPCOM00020250404el440038p)
- [10] [Trump Reciprocal Tariffs Take Effect, Upending U.S. Trade -- WSJ - Dow Jones Institutional News - 2025-04-09](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250409el4900055)
